In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
import pymongo

In [2]:
# Initializing a variable with the path for chromedriver
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# Creating a Browser instance for Google Chrome using chromedriver
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Initializing a variable with the url of the AAA website that displays fuel prices by grade by state
fuel_price_url = "https://gasprices.aaa.com/state-gas-price-averages/"
# Visiting fuel_price_url
browser.visit(fuel_price_url)

In [4]:
# Initializing variables that will hold the elements returned by scraping fuel_price_url for the names of each 
# state and their fuel prices by grade
states = browser.find_link_by_partial_href("https://gasprices.aaa.com?state=")
regular = browser.find_by_tag("td[class='regular']")
mid_grade = browser.find_by_tag("td[class='mid_grade']")
premium = browser.find_by_tag("td[class='premium']")

In [5]:
# Removing the first 10 entries in states because they are duplicate state names that do not have fuel data related  
# to them 
states = states[10:]

In [6]:
# Initializing a empty list's that will hold the names of each state and their fuel prices by grade
states_list = []
for state in states:
    states_list.append(state.text)
regular_list = []
for price in regular:
    regular_list.append(price.text)
mid_grade_list = []
for price in mid_grade:
    mid_grade_list.append(price.text)
premium_list = []
for price in premium:
    premium_list.append(price.text)

In [7]:
# Initializing a DataFrame with the lists of state names and their fuel prices by grade
state_fuel_df = pd.DataFrame({"state": states_list, "regular": regular_list, "mid_grade": mid_grade_list, "premium": premium_list})    

In [8]:
# Displaying state_fuel_df
state_fuel_df.head()

,state,regular,mid_grade,premium
0,Alaska,$3.077,$3.163,$3.301
1,Alabama,$2.306,$2.638,$2.955
2,Arkansas,$2.317,$2.600,$2.876
3,Arizona,$2.777,$3.011,$3.224
4,California,$3.612,$3.780,$3.891


In [9]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [10]:
# Define database and collection
db = client.fuel_prices_db
collection = db.fuel_prices

In [11]:
# Using a for-loop to iterate through the rows of state_fuel_df and insert each state's name and its fuel prices into 
# the MongoDB database fuel_prices_db collection fuel_prices as individual documents
for index, row in state_fuel_df.iterrows():

    post = {
                "state": row["state"],
                "regular": row["regular"],
                "mid_grade": row["mid_grade"],
                "premium": row["premium"]
            }

    collection.insert_one(post)
